In [2]:
import asyncio
from sqlalchemy.ext.asyncio import create_async_engine, AsyncSession
from sqlalchemy.orm import sessionmaker
from sqlalchemy import text

In [2]:
# Configuración de la base de datos
DATABASE_URL = "postgresql+asyncpg://franciscofurey@localhost/climatech"
engine = create_async_engine(DATABASE_URL, echo=True)
async_session = sessionmaker(engine, expire_on_commit=False, class_=AsyncSession)

async def show_tables_and_content():
    async with async_session() as session:
        # Obtener una lista de todas las tablas en la base de datos
        result = await session.execute(text("SELECT table_name FROM information_schema.tables WHERE table_schema='public'"))
        tables = result.scalars().all()
        
        for table in tables:
            print(f"--- Contenido de la tabla: {table} ---")
            # Mostrar las primeras 5 filas de cada tabla
            query = text(f"SELECT * FROM {table} LIMIT 5")
            result = await session.execute(query)
            rows = result.fetchall()
            for row in rows:
                print(row)
            print("\n")

# Ejecutar la función asíncrona en Jupyter
await show_tables_and_content()


2024-06-23 18:15:36,618 INFO sqlalchemy.engine.Engine select pg_catalog.version()


2024-06-23 18:15:36,618 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-06-23 18:15:36,620 INFO sqlalchemy.engine.Engine select current_schema()
2024-06-23 18:15:36,621 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-06-23 18:15:36,622 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2024-06-23 18:15:36,622 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-06-23 18:15:36,623 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-23 18:15:36,624 INFO sqlalchemy.engine.Engine SELECT table_name FROM information_schema.tables WHERE table_schema='public'
2024-06-23 18:15:36,624 INFO sqlalchemy.engine.Engine [generated in 0.00051s] ()
--- Contenido de la tabla: geography_columns ---
2024-06-23 18:15:36,632 INFO sqlalchemy.engine.Engine SELECT * FROM geography_columns LIMIT 5
2024-06-23 18:15:36,632 INFO sqlalchemy.engine.Engine [generated in 0.00046s] ()
('climatech', 'public', 'places', 'location', 2, 4326, 'Polygon')
('climatech', 'public', 'harmonized_landsat_sentinel_data', '

In [4]:
import asyncio
from sqlalchemy.ext.asyncio import create_async_engine, AsyncSession
from sqlalchemy.orm import sessionmaker
from sqlalchemy import text

# Configuración de la base de datos
DATABASE_URL = "postgresql+asyncpg://franciscofurey@localhost/climatech"
engine = create_async_engine(DATABASE_URL)
async_session = sessionmaker(bind=engine, expire_on_commit=False, class_=AsyncSession)

async def show_tables_row_count():
    """Muestra la cantidad de filas para cada tabla en la base de datos."""
    async with async_session() as session:
        # Obtener una lista de todas las tablas en la base de datos
        result = await session.execute(text("SELECT table_name FROM information_schema.tables WHERE table_schema='public'"))
        tables = result.scalars().all()
        
        for table in tables:
            # Contar las filas de cada tabla
            query = text(f"SELECT COUNT(*) FROM {table}")
            result = await session.execute(query)
            row_count = result.scalar()
            print(f"Tabla {table}: {row_count} filas\n")

# Ejecutar la función asíncrona en Jupyter
await show_tables_row_count()

Tabla geography_columns: 2 filas

Tabla geometry_columns: 0 filas

Tabla spatial_ref_sys: 8500 filas

Tabla places: 1 filas

Tabla harmonized_landsat_sentinel_data: 11934 filas

Tabla alembic_version: 1 filas



In [3]:
from sqlalchemy.ext.asyncio import create_async_engine, AsyncSession
from sqlalchemy.orm import sessionmaker

# Configuración de la base de datos
DATABASE_URL = "postgresql+asyncpg://franciscofurey@localhost/climatech"
engine = create_async_engine(DATABASE_URL)
async_session = sessionmaker(bind=engine, expire_on_commit=False, class_=AsyncSession)

async def delete_harmonized_data_by_place_id(place_id: int):
    """Elimina todos los registros en HarmonizedLandsatSentinelData basados en un Place id."""
    async with async_session() as session:
        # Primero, obtén todos los registros asociados con el place_id dado
        result = await session.execute(
            text(f"SELECT * FROM harmonized_landsat_sentinel_data WHERE place_id = :place_id"),
            {"place_id": place_id}
        )
        records = result.scalars().all()
        
        if records:
            # Si se encontraron registros, elimínalos
            await session.execute(
                text(f"DELETE FROM harmonized_landsat_sentinel_data WHERE place_id = :place_id"),
                {"place_id": place_id}
            )
            await session.commit()
            print(f"Todos los registros asociados al place_id {place_id} han sido eliminados.")
        else:
            print(f"No se encontraron registros para el place_id {place_id}.")

# Ejecuta la función con el ID de Place específico
await delete_harmonized_data_by_place_id(1)  # Reemplaza 123 con el ID real de Place que quieres eliminar


Todos los registros asociados al place_id 1 han sido eliminados.
